In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "at020-state-machine"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='5d950471-ac85-4847-8102-209d0a79b0a6', created_at=datetime.datetime(2025, 8, 13, 0, 15, 24, 722362, tzinfo=TzInfo(UTC)), name='93ec41a3-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 8, 13, 0, 15, 24, 722362, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='75379175-48cc-4d45-93f5-0e2125e6465d', agent_id='71580630-2e43-4db2-912f-124a80d8aca4', sequence_id=1, task_id='5d950471-ac85-4847-8102-209d0a79b0a6', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 8, 13, 0, 15, 26, 693593, tzinfo=TzInfo(UTC)))


In [5]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=5,
)

╭───────────────────────── USER [08/13/2025 00:15:26] ─────────────────────────╮
│ Hello tell me the latest news about AI and AI startups                       │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 5 seconds - returning collected messages


In [6]:
# Send a follow up  event to the agent in response to the agent's follow up question

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "I want to know what viral news came up and which startups failed, got acquired, or became very successful or popular in the last 3 months"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='f0bbedb7-95db-4708-acc0-e3094f34036a', agent_id='71580630-2e43-4db2-912f-124a80d8aca4', sequence_id=2, task_id='5d950471-ac85-4847-8102-209d0a79b0a6', content=TextContent(author='user', content='I want to know what viral news came up and which startups failed, got acquired, or became very successful or popular in the last 3 months', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 8, 13, 0, 15, 32, 633013, tzinfo=TzInfo(UTC)))


In [7]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=30, # Notice the longer timeout to give time for the agent to respond
)

╭───────────────────────── USER [08/13/2025 00:15:32] ─────────────────────────╮
│ I want to know what viral news came up and which startups failed, got        │
│ acquired, or became very successful or popular in the last 3 months          │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/13/2025 00:15:32] ─────────────────────────╮
│ Starting deep research process based on your query and follow-up             │
│ responses...                                                                 │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 30 seconds - returning collected messages
